## 2. Continuous time model: the geometric Brownian motion

In [ ]:
# import all stuff we will need
import numpy as np
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import scipy.stats as st

from numpy.random import randn
from gbm_d import gbm_d

### 2.1. Generating paths of a geometric Brownian motion

In [ ]:
n = 200; N = 10; # n number of time points, N number of paths
s0 = 2; sigma = 0.2; mu = 0.1; T = 1; # parameters of geometric Brownian motion

dt = T/n; tj = np.arange(dt,T+dt,dt); 
tj = np.reshape(np.asarray(tj),[n,1]); t = np.vstack((0.0,tj))    

Z = randn(N,n); X = np.cumsum(Z,axis=1)

s = s0*np.exp((mu-0.5*sigma**2)*tj+sigma*np.sqrt(dt)*X.T) # the stock prices at tj
s0 = np.repeat(s0,N); s = np.vstack((np.asarray(s0),s)) # the stock prices at tj, including s0

plt.plot(t,s); # plot all the paths
plt.plot(t,s0[0]*np.exp(mu*t),'k-'); # plot the expectation of St
plt.xlabel('$t$'); plt.ylabel('$S_t$');

### 2.2. Multivariate geometric Brownian motion

An "obvious way" to generalise the geometric Browian motion to multiple underlyings is to consider the $d$ stochastic differential equations

\begin{equation*}
{\rm d}S^i_t=\mu_iS^i_t{\rm d}t+\sigma_iS^i_t{\rm d}\widehat{W}^i_t,\quad S_0^i=s_i>0
\end{equation*}

with solution 

\begin{equation*}
S^i_t=s_ie^{(\mu_i-\frac{1}{2}\sigma_i^2)t+\sigma_i\widehat{W}^i_t}\;.
\end{equation*}

The Brownian motions $\widehat{W}^i_t$ are not independent but have the covariance/correlation matrix $\boldsymbol{\rho}=(\rho_{ij})$; details are given in the script.

The function <span style="color:orange">gbm_d.py</span> generates one path of $\textbf{S}_t:=(S_t^1,S_t^2,\ldots,S_t^d)$ for time points $t\in\{t_1,t_2,\ldots,t_n\}$. The function is described in the technical note in section 2.8 below. 

The function <span style="color:orange">gbm_d(s0,mu,Sigma,Tau,T,dt)</span> gives one path of a $d$-dimensional geometric Brownian motion, i.e.,

\begin{equation*}
S^i_t=s_ie^{(\mu_i-\frac{1}{2}\sigma_i^2)t+\sigma_i\widehat{W}^i_t}
\end{equation*}

at time points defined in $\mathcal{T} =\{t_1,\ldots,t_n\}$. If $\mathcal{T}$ is empty then $t_j = j\textrm{d}t$, $j=1,\ldots,\frac{T}{\textrm{d}t}$. 

For the construction of the correlated Brownian motions $\widehat{W}^i_t$, $i=1,\ldots,d$ see the script.

In [ ]:
# %load gbm_d.py
import numpy as np
from numpy.random import randn

def gbm_d(s0,mu,Sigma,Tau,T,dt):
    '''S,t = gbm_d(s0,mu,Sigma,Tau,T,dt) simulates one path of a d-dimensional 
       geometric Brownian motion 
       
       S = [S_0,S_t1,S_t2,...,S_T]
       
       at the time points specified in the list Tau = [t1,t2,...,T]. Each S_tj
       is a (row) array of length d (the GBM evaluated at tj).
       If Tau = [] is empty, the time points are evenly spaced, tj = j*dt, 
       with time step dt.
       
       s0 is a list of length d containing the initital values, mu is list of
       length d containing the drifts Sigma is the d-times-d covariance matrix.
       
       Example:
       s0 = [234,67]; mu = [0.037,0.027]
       Sigma = np.array([[0.18**2,0.18*0.15*0.68],[0.18*0.15*0.68,0.15**2]])
       S,t = gbm_d(s0,mu,Sigma,Tau,T,dt)
       plt.plot(S[:,0],S[:,1])
    '''
    
    if not Tau: # Tau is empty: we use evenly spaced time points in ]0,T]
        t = np.arange(dt,T+dt,dt);
    else: # Tau is not empty:
        t = Tau
    
    d = len(s0); n = len(t) # the dimensions
    
    # convert to arrays 
    s0 = np.reshape(np.asarray(s0),[1,d]) 
    mu = np.reshape(np.asarray(mu),[1,d])
    t = np.reshape(np.asarray(t),[n,1])
    t0 = np.vstack((0.0,t))
     
    L = np.linalg.cholesky(Sigma) # the Cholesky decomposition
    Z = randn(d,n); 
    sqrtDT = np.repeat(np.sqrt(np.diff(t0.T)),d,axis=0)

    X = np.cumsum(sqrtDT*Z,axis=1)
    A = np.repeat((mu-0.5*np.diag(Sigma)),n,axis=0)
    T = np.repeat(t,d,axis=1); S0 = np.repeat(s0,n,axis=0)
    S = S0*np.exp(A*T+np.matmul(L,X).T); S = np.vstack((s0,S));
    
    return S,t0

In [ ]:
s0 = [234,67]; mu = [0.037,0.027]
sigma = np.diag([0.18,0.15]); rho = np.array([[1,0.68],[0,1]]); 
rho = rho+rho.T-np.eye(2); Sigma = sigma.dot(rho).dot(sigma)
S,t = gbm_d(s0,mu,Sigma,[],1,1/(4*252))
plt.plot(S[:,0],S[:,1]);
plt.plot(s0[0],s0[1],'ko'); plt.plot(S[-1,0],S[-1,1],'ro'); # starting and ending point
plt.xlabel('$S_t^1$'); plt.ylabel('$S_t^2$');

Now, plot the graphs $t\mapsto S_t^i$.

In [ ]:
plt.plot(t,S); plt.xlabel('$t$'); plt.ylabel('$S_t$')
plt.legend(('stock 1','£stock 2'));

To better compare the two stocks, we normalize them, i.e., we consider the graphs $t\mapsto\frac{S_t^i}{S_0^i}$.

In [ ]:
plt.plot(t,S/np.asarray(s0)); plt.xlabel('$t$'); plt.ylabel('$S_t/S_0$')
plt.legend(('stock 1','stock 2'));

In [ ]:
s0 = [70,80,263]; mu = [0.03,0.05,0.02];
sigma = np.diag([0.1,0.2,0.4]); 
rho = np.array([[1,0.8,-0.5],[0,1,-0.7],[0,0,1]]); 
rho = rho+rho.T-np.eye(3); Sigma = sigma.dot(rho).dot(sigma)
S,t = gbm_d(s0,mu,Sigma,[],1,1/(2*252))

ax = plt.axes(projection='3d')

ax.scatter3D(S[0,0],S[0,1],S[0,2],c='k',marker='.',s=100); # starting point is black
ax.scatter3D(S[-1,0],S[-1,1],S[-1,2],c='r',marker='.',s=100); # ending point is red
ax.plot3D(S[:,0],S[:,1],S[:,2]);
ax.set_xlabel('$S_t^1$'), ax.set_ylabel('$S_t^2$'), ax.set_zlabel('$S_t^3$');

In [ ]:
plt.plot(t,S/np.asarray(s0)); plt.xlabel('$t$'); plt.ylabel('$S_t/S_0$')
plt.legend(('stock 1','stock 2','stock 3'));